In [42]:
# [1]包导入              # XGBT和LGBT的对比
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import missingno as msno
import warnings
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from scipy.stats import uniform, randint
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import randint, uniform
import numpy as np
import pandas as pd


In [43]:
# [2]全局变量
TARGET='SalePrice'
oTARGET='SalePrice'
# 设置支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体支持中文
plt.rcParams['axes.unicode_minus'] = False    # 正常显示负号
# 屏蔽此类警告
import warnings
warnings.filterwarnings("ignore", message="Could not find the number of physical cores")
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # 或其他你机器的逻辑核心数

In [44]:
# [3]数据集读取
otrain_data = pd.read_csv('train.csv')
otest_data = pd.read_csv('test.csv')
train_data = otrain_data.copy()
test_data = otest_data.copy()         # 原始数据备份
train_data.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [45]:
# [5]筛选训练集和测试集都存在的列
cols = [TARGET]         # target列一定保留
for c in train_data.columns:
    if c in test_data.columns:
        cols.append(c)
train_data = train_data[cols]
test_data = test_data[cols[1:]]        # test_data不包含结果列

In [46]:
X_train = train_data.drop(TARGET, axis=1)
y_train = train_data[TARGET]
X_test = test_data

X_train_split, X_valid, y_train_split, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

In [47]:
# 1. 自动识别所有非数值列（object 或 category 类型）
cat_cols = X_train_split.select_dtypes(include=['object', 'category']).columns.tolist()

# 2. 对训练集、验证集、测试集进行独热编码
X_train_encoded = pd.get_dummies(X_train_split, columns=cat_cols, drop_first=True)
X_valid_encoded = pd.get_dummies(X_valid, columns=cat_cols, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

# 3. 对齐验证集和测试集的列，使其与训练集完全一致
X_valid_encoded = X_valid_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)


In [48]:
xgb_param_dist = {
    'n_estimators': randint(100, 2000),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(2, 10),
    'min_child_weight': uniform(1, 6),
    'gamma': uniform(0, 0.5),
    'subsample': uniform(0.3, 0.7),
    'colsample_bytree': uniform(0.3, 0.7),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
}

lgb_param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.2),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'reg_alpha': uniform(0.0, 1.0),
    'reg_lambda': uniform(0.0, 1.0)
}


In [49]:
# 自定义评估函数：log RMSE（与前面一致）
def rmse_log_score(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

# 包装为 scorer，越小越好，需加 greater_is_better=False
rmse_scorer = make_scorer(rmse_log_score, greater_is_better=False)

In [37]:
xgb = XGBRegressor(random_state=42, tree_method='hist')
# 随机搜索（可调 n_iter 次数）
xgb_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=xgb_param_dist,
    n_iter=50,
    scoring=rmse_scorer,
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)
xgb_search.fit(X_train_encoded, y_train_split)
best_xgb = xgb_search.best_estimator_

lgb = LGBMRegressor(random_state=42)
lgb_search = RandomizedSearchCV(
    estimator=lgb,
    param_distributions=lgb_param_dist,
    scoring='neg_root_mean_squared_error',
    n_iter=30,
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)
lgb_search.fit(X_train_encoded, y_train_split)
best_lgb = lgb_search.best_estimator_


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.7min finished


Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    5.8s finished


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000977 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3402
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 157
[LightGBM] [Info] Start training from score 181441.541952
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

In [50]:
# 预测
preds_xgb = best_xgb.predict(X_test_encoded)
submission_xgb = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': preds_xgb
})
submission_xgb.to_csv('submission_xgb.csv', index=False)

preds_lgb = best_lgb.predict(X_test_encoded)
submission_lgb = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': preds_lgb
})
submission_lgb.to_csv('submission_lgb.csv', index=False)


In [52]:
for name, model in {'XGBoost': best_xgb, 'LightGBM': best_lgb}.items():
    preds = model.predict(X_valid_encoded)
    log_rmse = np.sqrt(mean_squared_error(np.log1p(y_valid), np.log1p(preds)))
    print(f'{name} 验证集上的对数 RMSE: {log_rmse:.4f}')

XGBoost 验证集上的对数 RMSE: 0.1429
LightGBM 验证集上的对数 RMSE: 0.1401
